# Final Project Plan
__`Team 4-1`__

## Phase Leader Plan
| Phase |  Phase Leader |
|:---:|:---:|
| **Phase 0, HW5**: Finalize Teams, and submitting HW5 | Danielle Yoseloff |
| **Phase 1**: Project Plan, describe datasets, joins, tasks, and metrics  | Mohamed Bakr |
|**Phase 2**: EDA, baseline pipeline, Scalability, Efficiency, Distributed/parallel Training, and Scoring Pipeline| Shruti Gupta |
|**Phase 3**: Select the optimal algorithm, fine-tune and submit a final report| Erica Landreth |

%md
## Credit assignment plan

| Phase | team Meamber | Tasks | Hrs|
|:---:|:---:|:---:|:---:|
|**PHASE 0**| Danielle Yoseloff | Forming Team, Create Slack Channel, and team introduction |  2 |
|**PHASE 1**| Danielle Yoseloff | Machine algorithms and metrics | 8 |
|| | Pipeline Graph | 1 |
||Erica Landreth | Abstract and Report Editing | 3 |
||| EDA | 2.5 |
||| Data Description | 8 |
|| Shruti Gupta | EDA | 4 |
|| |Missing & Null Value Exploration | 4 |
|| Mohamed Bakr | Phase Leader Table, Credit Assigment plan, and GANTT chart |  8 |
||| Digesting the Data and Checkpointing Strategy | 4 |
|||Report editing and review| 2 |
|**PHASE 2**| Danielle Yoseloff | | |
||Erica Landreth | |  |
|| Shruti Gupta | | |
|| Mohamed Bakr | | |
|**PHASE 3**| Danielle Yoseloff | | |
||Erica Landreth | | |
|| Shruti Gupta | | |
|| Mohamed Bakr | | |


**Detailed Plan and GANTT Chart:** https://docs.google.com/spreadsheets/d/1E4A3SaTAEjh9owH4SBUMv987bktwrW4Q6TXCZ5LJ6Xg/edit?usp=sharing

**Note:** Phase 2 and 3 plans are tentative and subject to change.

## Abstract

The United States (US) is home to one of the busiest and most complex airspaces in the world. With an average of 45,000 flights taking off or landing at US airports each day [1], effective organization of resources at those airports is crucial to achieve safe, efficient, and continuous air traffic operations nationwide. When flight departures are delayed, airport employees must reallocate spatial resources, such as gate and runway space, and personnel resources; such as gate agents, air traffic controllers, air marshallers, and ground crew; to accommodate the altered schedule. With this airport resource management business case in mind, we aim to design a machine learning model to predict whether or not a given flight is delayed two hours before its scheduled departure, in order to provide airport employees the information they need to schedule resources in a timely fashion.

To do so, we will use historic Department of Transportation (DoT) flight data and associated National Oceanic and Atmospheric Association (NOAA) weather station reports from the years 2015 to 2021 to train machine learning models predicting departure delay status based on flight metadata and weather conditions. We will first establish baseline performance using a logistic regression model, then will train more advanced models using random forest, XGBoost, multi-layer-perceptron neural networks, and graph neural networks. To evaluate the performance of our models, we will use F1 score and area under the curve (AUC).

## Research Objective

Our primary customer is the airport management and administration. Based on our business case, we use the FAA definition of delay, which applies to a flight that departs 15 minutes or more after its scheduled departure; we assume that 15 minutes of delay defines the cutoff after which airport resources may need to be reallocated. We may also consider cancelled flights to be delayed based on our business scenario. Our objective is to use machine learning classification models to predict departure delay status based on flight,airport, and weather data.

## Data Description

### Data size and source

For our modeling and analysis, we will combine data from the following four sources:

* **Flights:** DoT historical flight data from the years 2015-2021 (2015 Q1 sample: 2,806,942 rows by 109 columns)
* **Weather:** NOAA weather conditions for the corresponding time period (2015 Q1 sample: 30,528,602 rows by 124 columns)
* **Stations:** Weather stations metadata dataset, including mapping to nearest airport (5,004,169 rows by 12 columns)
* **Airports:** Airport metadata dataset (57,421 rows by 12 columns)

We will use the above bolded names to refer to the datasets for the remainder of the report.


### Data dictionary

This section defines the variables from each source that we plan to use for our modeling and analysis.

#### Flights data

The flights data provide metadata for a given flight, and will also help us to study time-series trends and aggregate delay statistics by characteristics such as airport and airline. The below definitions were informed by DoT documentation [4].

| Column |  Raw Data Type | Meaning | Intended Use |
|:---:|:---:|:---:|:---:|
| QUARTER | Integer | Quarter | Time series feature engineering |
| MONTH | Integer | Month | Time series feature engineering |
| DAY_OF_WEEK | Integer | 1-7: Monday-Sunday | Create day-of-week categorical variable, to capture week-periodic trends |
| FL_DATE | String | Flight date | Time series feature engineering |
| OP_UNIQUE_CARRIER | String | Recommended (from DoT documentation) carrier ID for multi-year analysis | Create airline categorical variable |
| TAIL_NUM | String | Aircraft tail number (registration code) | Follow delays of an individual aircraft |
| OP_CARRIER_FL_NUM | Integer | Flight number, specific to a carrier | Combine with carrier info to crate categorical flight route feature |
| ORIGIN_AIRPORT_ID | Integer | Recommended (from DoT documentation) airport ID for multi-year analysis | Create origin airport categorical variable |
| ORIGIN | String | Origin airport IATA code | Join to airports data |
| DEST_AIRPORT_ID | Integer | Recommended (from DoT documentation) airport ID for multi-year analysis | Create destination airport categorical variable |
| DEST | String | Destination airport IATA code | Join to airports data |
| CRS_DEP_TIME | Integer | Scheduled departure time (local, HHMM format) | Capture time-of-day periodic trends; filter time-relevant records from other data sources |
| DEP_TIME | Integer | Actual departure time (local, HHMM format) | Characterize airport traffic flow over time |
| DEP_DELAY | Double | Departure delay (min) | Define Boolean departure delay status |
| TAXI_OUT | Double | Time taxiing out (min) | Characterize airport traffic flow |
| WHEELS_OFF | Integer | Time flight took off (local, HHMM format) | Characterize airport traffic flow |
| WHEELS_ON | Integer | Time flight landed (local, HHMM format) | Characterize airport traffic flow |
| TAXI_IN | Double | Time taxiing in (min) | Characterize airport traffic flow |
| CRS_ARR_TIME | Integer | scheduled arrival time (local, HHMM format) | Capture time-of-day periodic trends; filter time-relevant records from other data sources |
| ARR_TIME | Integer | Actual arrival time (local, HHMM format) | Characterize airport traffic flow over time |
| ARR_DELAY | Double | Arrival delay (min) | Characterize arrival delays |
| CANCELLED | Double | 1.0/0.0: Cancelled/not cancelled | Define Boolean cancellation status |
| DIVERTED | Double | 1.0/0.0: Diverted/not diverted | Define Boolean diversion status |
| CRS_ELAPSED_TIME | Double | Scheduled flight duration (min) | Represent flight length for flights within 2-hour window|
| ACTUAL_ELAPSED_TIME | Double | Actual flight duration (min) | Represent flight length for flights outside of 2-hour window |
| AIRTIME | Double | Time between take-off and landing (min) | Characterize air traffic flow |
| DISTANCE | Double | Distance between origin and destination airports | Represent flight length |
| YEAR | Integer | Year | Time series feature engineering |

We chose to drop some variables from the full flights table based on redundancy, the proportion of missing values, and relevance to our analysis. These include alternate representations of airport and airline ID's and diversion information.


#### Weather data

The weather data allows us to define weather conditions relevant to an individual flight, as well as characterize longer-term regional weather trends. The below definitions were informed by NOAA documentation [2] and [3].

| Column |  Raw Data Type | Meaning | Intended Use |
|:---:|:---:|:---:|:---:|
| STATION | String | Weather station ID | Key for joining to stations data |
| DATE | String | Date the time (UTC) of weather report | Filter weather reports in time |
| YEAR | Int | Year | Time series feature engineering |
| LATITUDE | String | Station latitude (degrees North) | Characterize station location |
| LONGITUDE | String | Station longitude (degrees East) | Characterize station location |
| ELEVATION | String | Station elevation above sea level | Characterize station location |
| REPORT_TYPE | String | Weather report type | Filter to relevant report types |
| SOURCE | String | Source(s) contributing to weather report | Filter to relevant source types |
| HourlyAltimeterSetting | String | Atmospheric pressure (in Hg) | Define weather conditions |
| HourlyDewPointTemperature | String | Dew point temperature (degrees F) | Define weather conditions |
| HourlyDryBulbTemperature | String | Air temperature (degrees F) | Define weather conditions |
| HourlyPrecipitation | String | Precipitation amount (in) | Define weather conditions |
| HourlyPresentWeatherType | String | String code defining present weather *e.g.* rain or hail | Define weather conditions |
| HourlyPressureChange | String | Change in pressure (in Hg) | Define weather conditions |
| HourlyPressureTendency | String | Categorical characterization of recent pressure change | Define weather conditions |
| HourlyRelativeHumidity | String | Relative humidity (percentage) | Define weather conditions |
| HourlySkyConditions | String | Cloud conditions for up to 3 cloud layers | Define weather conditions |
| HourlySeaLevelPressure | String | Sea level pressure (in Hg) | Define weather conditions |
| HourlyStationPressure | String | Station pressure (in Hg) | Define weather conditions |
| HourlyVisibility | String | Horizontal visibility (mi) | Define weather conditions |
| HourlyWetBulbTemperature | String | Wet bulb temperature (degrees F) | Define weather conditions |
| HourlyWindDirection | String | Wind direction (Degrees clockwise from true North) | Define weather conditions |
| HourlyWindGustSpeed | String | Wind gust speed (mph) | Define weather conditions |
| HourlyWindSpeed | String | Wind speed (mph) | Define weather conditions |

We chose to drop some variables from the full weather table based on redundancy, the proportion of missing values, and relevance to our analysis. These include alternate station identifiers, daily and monthly averages, and station backup/maintenance information.

#### Weather station data

The weather station data defines the distances from various weather stations to various airports.

| Column |  Raw Data Type | Meaning | Intended Use |
|:---:|:---:|:---:|:---:|
| station_id | String | Weather station ID | Key for joining to weather data |
| lat | Double | Station latitude (degrees North) | Characterize station location |
| lon | Double | Station longitude (degrees East) | Characterize station location |
| neighbor_name | String | Airport name | Sanity check for joins |
| neighbor_state | String | Airport state | Sanity check for joins |
| neighbor_call | String | Airport ICAO code | Key for joining to airport data |
| neighbor_lat | Double | Airport latitude (degrees North) | Characterize airport location |
| neighbor_lon | Double | Airport longitude (degrees East) | Characterize airport location |
| distance_to_neighbor | Double | Distance (mi) from station to airport | Find weather stations near a given airport |


We chose to drop some variables from the full stations table based on redundancy and relevance to our analysis. These include alternate station and airport identifiers.

#### Airport data

The airport dataset provides airport metadata and identifiers necessary for joins.

| Column |  Raw Data Type | Meaning | Intended Use |
|:---:|:---:|:---:|:---:|
| ident | String | Airport ICAO code | Join to stations data |
| type | String | Airport type | Characterize airport operations |
| name | String | Airport name | Sanity check for joins |
| continent | String | Airport continent | Filtering |
| iso_country | String | ISO code of airport country | Filtering |
| iso_region | String | ISO code of airport region | Filtering and sanity check for joins |
| iata_code | String | Airport IATA code | Join to flights data |
| coordinates | String | Airport latitude and longitude | Characterize airport location |

We chose to drop some variables from the full airports table based on redundancy and relevance to our analysis. These include alternative identifiers and local, categorical location codes.

### Joining strategy

To identify relevant weather information for a given flight, we will need to join data from all four tables. The following outlines our joining strategy:
* For each flight, add airport metadata for the origin and destination airports, using IATA airport codes. To do so, we join flights and airports using the flights ORIGIN field as primary key and airports iata_code as foreign key. We rename the airports fields to indicate that they pertain to the origin airport. Then we repeat the process for the destination airport, joining flights and airports using flights DEST field as primary key and airports iata_code as foreign key, this time renaming the airports field to indicate that they pertain to the destination airport. This provides (among other data) the ICAO codes for the origin and destination airports.
* Use the distance_to_neighbor field of the stations table to filter the stations data to contain only data for the closest weather station to each airport. Then join the flights data and the filtered stations data for both the origin and destination airport using the flight data origin ICAO and destination ICAO codes as primary keys, respectively, and the airports neighbor_call field as foreign key. This provides (among other data) the station ID for the closeset weather station for the origin and destination airports.
* Using the relevant weather station ID identified for each airport, incorporate the most recent weather data that was recorded at least two hours ahead of a given flight's departure. During Phase II, we will explore an appropriate time window scheme, but the intent is to create "current condition" variables from recent records at the closest weather station to each airport, and join that back to the airports table.

### Leakage strategies

There are two main areas where we'll need to be careful about data leakage: feature engineering and splitting our data for training, cross-validation, and test.

To avoid leakage during feature engineering, we will not include any data from within 2 hours of a flight's scheduled departure in order to calculate features for that flight. This means that any weather related features will be calculated using weather station data that is at least two hours stale. Any engineered features related to recent delays for a given airport, airline, or other characteristics will only be calculated using data for which actual delay data was available two hours ahead of the given flight's scheduled departure. We will also normalize using the training data so that the model does not learn any information about the test data.

We have a few possible strategies to avoid leakage in our data splits. Firstly, we propose regular time-series cross validation in which the test split is always ahead of the training split. For example, for 1 year's worth of data, we will use the following split:
* CV split 1: Train on January through March, test on April
* CV split 2: Train on January through April, test on May, ... et cetera.

However, since this strategy might introduce leakage, allowing the model from observe future data and memorize patterns, our second proposal would be to use blocked time-series cross-validation.


### Digesting the Data and Checkpointing Strategy

To maintain consistency and prevent redundant processing, we treat our data analysis pipeline as a directed acyclic graph (DAG), implementing checkpoints at key stages. We are adopting some of the ideas from the Cookiecutter for Data Science project by drivendata.org, as outlined in this guide: [Link](https://cookiecutter-data-science.drivendata.org/using-the-template/). This structured approach ensures reproducibility, scalability, and organization throughout the data pipeline.

---

#### Data Ingestion:

* **Purpose:** Load raw data while ensuring data integrity and reproducibility.
* **Process:**
  - Raw flight, weather, and airport datasets are ingested from the directory: `dbfs:/mnt/mids-w261/`
  - The ingested datasets are stored in the raw data directory:`dbfs:/student-groups/Group_4_1/raw`
  - Spark is leveraged for distributed data handling to efficiently process large datasets (e.g., multi-year flight records).

---

#### Checkpointing Strategy:

* **Purpose:** Enhance efficiency, ensure fault tolerance, and prevent re-computation by storing intermediate data at key stages. Each checkpoint serves a distinct purpose, ensuring smooth transitions between phases.

* **Process:**
  1. **Post-Ingestion:**  
    The ingested datasets will stored in Parquet format in the `interim` directory for use in the next steps:`dbfs:/student-groups/Group_4_1/interim`

  2. **Exploratory Data Analysis (EDA):**  
    During this Step, the data is read the prepared Parquet files directly from the `interim` directory created in the previous step.

  3. **Data joining:**
    After performing the necessary joins between datasets usign the the prepared Parquet files/, the finalized data will be stored in the `interim` directory with the `joined_` prefix.

  4. **Train/Validate/Test Split:**  
    In this step we will only read the joined dataset from the `interim` directroy then we will partition the dataset before the feature engineering step to ensure proper evaluation and prevent data leakage.
      - We will perform time-based split the feature-engineered dataset into training, validation, and test sets.  
      - Each split will be stored in the `interim` directory with appropriate prefixes:  
        ```plaintext
        dbfs:/student-groups/Group_4_1/interim/train_xxx.parquet
        dbfs:/student-groups/Group_4_1/interim/val_xxx.parquet
        dbfs:/student-groups/Group_4_1/interim/test_xxx.parquet
        ```

  5. **Feature Engineering:**  
    In this step we will develop and integrate custom features while ensuring version control by following these steps:
      - Load training dataset from the `interim` directory.  
      - Engineer relevant features, ensuring no data leakage occurs by avoiding the use of future data in the feature creation process.  
      - After feature engineering, store the enriched dataset in the `processed` directory, appending `_v(x)` to track versions.  
      - Optionally, add the model name in the filename to align feature sets with specific models:  
        ```plaintext
        e.g.: dbfs:/student-groups/Group_4_1/processed/FE_v1_RF.parquet
        ```
      - After fitting our feature engineering pipeline on the training data, we will apply the same transformations to the validation and test data to create the corresponding features using a pipeline to automate the feature engineering and model training process, ensuring consistency and reproducibility.

  6. **Modeling:**  
    To ensure consistency only feature engineered datasets is loaded from the `processed` directory for the modeling stage.  

  ---

  #### Metadata Tracking (Optional):

  - **Purpose:** Enhance reproducibility by tracking key details at each checkpoint.  
  - **Process:** At each checkpoint, generate a metadata file (e.g., JSON) recording:  
    - Timestamp of creation  
    - Applied transformations (joins, cleaning steps, feature creation)  
    - Version number  
    - Count of records and features  
    - Hash of the dataset to detect changes  

    **Example:**
    ```json
    {
      "checkpoint": "FE_ready_v1",
      "timestamp": "2025-03-14T12:00:00Z",
      "transformations": ["Join: Flights + Weather", "Imputation: Missing Arrivals"],
      "num_records": 1_500_000,
      "num_features": 23,
      "hash": "abc123def456"
    }
    ```

### Quick and dirty EDA

First, we explore the continuous distribution of flight delays, where a delay is defined by the FAA as departing 15 minutes or more past its scheduled time. We use the column DEP_DELAY which is defined as the difference in minutes between scheduled and actual departure time, and the column DEP_DEL15 which is a delay indicator. 80% of flights depart without delays, and of these, a majority are actually reported as departing early. The interquartile range of the continuous representation ranges between 5 minutes early and 9 minutes late, further showing the tendency across all flights to depart early or on time.

Of the 20% of flights that depart delayed, 7.5% have an origin of Chicago O'Hare, 5.7% have an origin of Atlanta Hartsfield-Jackson and 5.5% have an origin of Dallas Fort Worth. The airline with the greatest percentage of its flights flagged as delayed is Frontier at around 30%, and the airine with the least share of its flights delayed is Hawaiian Airlines at around 9%. However, this could be due to a difference in the number routes flown by each airline. For example, no Hawaiian Airlines flights as reported as originating from O'Hare in this time period. We plan to explore more about the characteristics of delayed flights in the Phase II EDA.

We may consider flight cancellations as functionally analogous to long-term delays, similar to those reported as exceeding 24 hours. This approach is based on the idea that cancellations, like long delays, can disrupt resource allocation and operational flow. By treating cancellations as delays, we hope to uncover valuable patterns in the data that could improve model performance. However, this is a preliminary strategy that will be further explored during Phase II EDA. Depending on the insights we see there, we may decide to exclude these cancellations, i.e. the  departure delay null records, from our analysis. If we choose to exclude the 3% of null values representing cancellations, we would also remove the 0.08% of flights that are both flagged as canceled and reported with delay times, as these involve time spent away from the gate.

<center>

|                | Delayed | Not Delayed |
|:--------------:|:-------:|:-----------:|
| Cancellations Included |   20.4%    |      79.6%      |
| Cancellations Excluded |   22.8%    |      77.2%      |

</center>

A very small share of delayed flights are reported as having been diverted. We do not plan to count these diverted flights as delayed because they did not return to the gate at the origin airport (unlike a cancelled flight), and instead land at an alternate airport. Diversions also represent a very small share of overall flights, so we may not include the  associated features about each diversion.

### Missing value plan and analysis

In the flights dataset, the TAIL_NUM column is essential for relate multiple flights by the same aircraft and engineering graph-related features. Less than 1% of the values are missing, so we plan to fill in the null values with a missing value indicator which will be inherited by dependent features. The columns attributing delay minutes to causes (carrier, NAS, weather, security, late aircraft) are missing 80% values for delayed flights, so we plan to drop them. Some time-related columns appear to have many missing values (ARR_TIME, ELAPSED_TIME, WHEELS_ON, WHEELS_OFF, et cetera) but further analysis shows that these columns only contain null values if the flight is cancelled or diverted, so we do not need to conduct any interpolation based on records with similar flight profiles and conditions.

In the weather dataset, some daily, monthly, or long-term aggregate statistics contain too many null values for our usage (i.e., monthly dewpoint temperature or total snowfall). We may need to aggregate our own features if we find that an aggregate level estimate is important. For more granular level features like hourly precipitation or visibility, we will determine what columns are the priority to include based on the Phase II EDA, then use time series techniques like moving average or spline interpolation to fill the gaps in the data.

### Sampling Strategy

The computations of datapoint distances used in oversampling/SMOTE is not favorable considering the size of our data, so we plan to use undersampling to address the class imbalance in our outcome variable. Because we will need to undersample drastically to reach a class balance we will we be intentional about how we sample to ensure vairables of interest are not disproportionally influenced due to the time seiries quality of the data. This may look like making sure every airport or every day is still represented, but plan to explore these strategies further during Phase II.

## Machine algorithms and metrics.

We have chosen to pursue a binary classification problem using the formal DoT definition of delay, DepDel15, as our dependent variable. This best suits our use case because our audience, flight traffic control, uses this definition to dictate procedure.

We are interested in four potential models: Majority Class, Logistic Regression, XGBoost, and Random Forest. Majority class can be used as a most basic baseline model because it is the current operating assumption. We would also like to create a logic regression as an alternative baseline model. For more advanced methods we would like to create both XGBoost and Random forest models. The loss function for all models is a log loss/ cross entropy loss. 

Our metrics of interest will be: AUC, ROC graphs, Accuracy, Precision, Recall, and F-Score. We believe the most useful metrics will be AUC and F-Score because they provide a balanced perspective of false positives and false negatives. In our use case true positives indicate a flight is really delayed while true negatives indicate a flight is really on time.

**Type I error** (*FP:* The model predicts a delay, but the flight departs on time) this may cause confusion and unnecessary changes from air traffic control. however, it's more acceptable if we are prioritizing caution and want to minimize unexpected delays.

**Type II error** (*FN:* The model predicts the flight will be on time, but it ends up delayed) Passengers and crew aren’t prepared for the delay, potentially leading to missed connections, poor customer satisfaction, and operational disruptions. This type is more costly if unexpected delays cause major disruptions, and you aim to avoid them at all costs.

We will also consider Gini Impurity for the Random Forest model hypertuning. 


#### Logistic Function
$$\sigma(t) = \frac{1}{1+\exp(-t)}$$

#### Binomial Cross Entropy Loss Function
$$J(\theta) = -\frac{1}{m}\sum_{i=1}^{m}[y^{(i)}\log(\hat{p}^{(i)})+(1-y^{(i)})\log(1-\hat{p}^{(i)}))]$$

#### Calssification Metircs
$$F1-Score = \frac{2 * Precision * Recall}{Precision + Recall}$$

## Machine Learning Pipelines

The following steps define our machine learning pipeline:
1. **Ingestion:** Load raw data into Spark DataFrames
2. **Feature Selection:** Drop unnecessary columns
3. **Join:** Combine data sources into joined DataFrame
4. **Split:** Divide data into training, validation, and test splits
5. **Sample:** Undersampling applied to training split
6. **Imputation:** Fill missing values based on training data
7. **Feature engineering:** Add engineered features
8. **Hyperparameter tuning:** Use cross-validation to train model that balances performance and generalizability
9. **Model training:** Train model using chosen hyperparameters
10. **Model evaluation:** Assess trained models on test data

<div style="text-align: center; line-height: 10; padding-top: 30px;  padding-bottom: 30px;">
<img src="https://github.com/bakr-UCB/261-Final-Project/blob/main/reports/figures/ml_pipeline.png?raw=true" alt="ML Pipeline" style="width: 500px">
<div>

## Deliverables

### Final Project Report (Databricks Notebook)
  - A comprehensive Jupyter Notebook documenting the full pipeline, from data preprocessing to model evaluation.
  - Clear explanations of each step, including feature engineering, model experimentation, and hyperparameter tuning.
  - Visualizations and metrics to compare model performance, highlighting key findings and insights.
  - Discussion of challenges faced, lessons learned, and recommendations for future improvements.

### In Person Presentaion
  - A concise and visually engaging presentation summarizing the project journey, key findings, and final model selection.
  - Focused sections on feature engineering, model evaluation, and the impact of selected features.
  - Presentation of results with clear narratives, supported by charts and tables, to effectively communicate insights to a broad audience.
  - Concluding remarks covering the significance of the results and potential next steps.

## Bibliography

<ol>
    <li>"Air Traffic." Federal Aviation Administration, 20 Apr. 2023, www.faa.gov/air_traffic.</li>
    <li>"Federal Climate Complex Data Documentation for Integrated Surface Data (ISD)." NOAA NCEI, 12 Jan. 2018, https://www.ncei.noaa.gov/data/global-hourly/doc/isd-format-document.pdf. Accessed 16 Mar. 2025.</li>
    <li>“Local Climatological Data (LCD) Dataset Documentation.” Local Climatological Data (LCD) Data, NOAA NCEI, www.ncei.noaa.gov/pub/data/cdo/documentation/LCD_documentation.pdf. Accessed 16 Mar. 2025.</li>
    <li>"Reporting Carrier On-Time Performance (1987-present)." Bureau of Transportation Statistics, https://www.transtats.bts.gov/Fields.asp?gnoyr_VQ=FGJ. Accessed 16 Mar. 2025.</li>
</ol>